# Binary encoding

In [18]:
# Función para convertir binario a Gray code
def binary_to_gray(binary_str):
    return binary_str[0] + ''.join(str(int(binary_str[i-1]) ^ int(binary_str[i])) for i in range(1, len(binary_str)))

# Función para convertir Gray code a binario
def gray_to_binary(gray_str):
    binary_str = gray_str[0]
    for i in range(1, len(gray_str)):
        binary_str += str(int(binary_str[i - 1]) ^ int(gray_str[i]))
    return binary_str

# Diccionarios para codificación/decodificación
layer_type_options = {
    'Conv2D': '000', 
    'BatchNorm': '001', 
    'MaxPooling': '010', 
    'Dropout': '011', 
    'Dense': '100', 
    'Flatten': '101',
    'DepthwiseConv2D': '110',  # Nueva capa Depthwise Conv2D
    'GlobalAveragePooling2D': '111'  # Nueva capa GlobalAveragePooling2D
}
filter_options = {32: '00', 30: '01', 16: '10', 8: '11'}
stride_options = {1: '0', 2: '1'}
dropout_options = {0.2: '00', 0.3: '01', 0.4: '10', 0.5: '11'}
neuron_options = {256: '00', 128: '01', 32: '10', 1: '11'}
activation_options = {'relu': '00', 'leaky_relu': '01', 'sigmoid': '10', 'tanh': '11'}

# Función para codificar parámetros de las capas
def encode_layer_params(layer_type, filters=None, strides=None, dropout=None, neurons=None, activation=None):
    binary_representation = layer_type_options.get(layer_type, '000')
    
    if layer_type == 'Conv2D':
        binary_representation += filter_options.get(filters, '00')
        binary_representation += stride_options.get(strides, '0')
        binary_representation += activation_options.get(activation, '00')
    
    if layer_type == 'DepthwiseConv2D':
        filters_binary = filter_options.get(filters, '00')
        strides_binary = stride_options.get(strides, '0')
        activation_binary = activation_options.get(activation, '00')
        return layer_type_options['DepthwiseConv2D'] + filters_binary + strides_binary + activation_binary
    
    elif layer_type == 'GlobalAveragePooling2D':
        return layer_type_options['GlobalAveragePooling2D']
    
    elif layer_type == 'MaxPooling':
        binary_representation += stride_options.get(strides, '0')
    
    elif layer_type == 'Dropout':
        binary_representation += dropout_options.get(dropout, '00')
    
    elif layer_type == 'Dense':
        binary_representation += neuron_options.get(neurons, '00')
        binary_representation += activation_options.get(activation, '00')
    
    return binary_representation

# Función para decodificar parámetros de las capas
def decode_layer_params(encoded_gray):
    encoded_binary = gray_to_binary(encoded_gray)
    layer_type = encoded_binary[:3]
    
    if layer_type == '000':  # Conv2D
        return {
            'type': 'Conv2D',
            'filters': decode_value(encoded_binary[3:5], filter_options, 32),
            'kernel_size': (3, 3),
            'strides': decode_value(encoded_binary[5], stride_options, 1),
            'padding': 'same',
            'activation': decode_value(encoded_binary[6:8], activation_options, 'relu')
        }
    
    elif layer_type == '001':  # BatchNorm
        return {'type': 'BatchNorm'}
    
    elif layer_type == '010':  # MaxPooling
        return {'type': 'MaxPooling', 'strides': decode_value(encoded_binary[3], stride_options, 1)}
    
    elif layer_type == '011':  # Dropout
        return {'type': 'Dropout', 'rate': decode_value(encoded_binary[3:5], dropout_options, 0.2)}
    
    elif layer_type == '100':  # Dense
        # Corregir la decodificación de 'units'
        return {
            'type': 'Dense',
            'units': decode_value(encoded_binary[3:5], neuron_options, 256),  # Ajustamos para corregir 'units'
            'activation': decode_value(encoded_binary[5:7], activation_options, 'relu')
        }
    
    elif layer_type == '101':  # Flatten
        return {'type': 'Flatten'}
    
    elif layer_type == '110':  # DepthwiseConv2D (nueva capa)
        return {
            'type': 'DepthwiseConv2D',
            'filters': decode_value(encoded_binary[3:5], filter_options, 32),  # Reutilizamos los filtros
            'kernel_size': (3, 3),  # Valor constante
            'strides': decode_value(encoded_binary[5], stride_options, 1),
            'padding': 'same',  # Valor constante
            'activation': decode_value(encoded_binary[6:8], activation_options, 'relu')
        }
    
    elif layer_type == '111':  # GlobalAveragePooling2D (nueva capa)
        return {'type': 'GlobalAveragePooling2D'}
    
    else:
        raise ValueError("Tipo de capa desconocido.")
# Función auxiliar para decodificar valores de las opciones
def decode_value(bits, options_dict, default_value):
    return {v: k for k, v in options_dict.items()}.get(bits, default_value)

# Ejemplo de uso optimizado
binary_conv2d = encode_layer_params('Conv2D', filters=32, strides=1, activation='relu')
gray_conv2d = binary_to_gray(binary_conv2d)
decoded_conv2d = decode_layer_params(gray_conv2d)
print(f"Binary encoding of Conv2D: {binary_conv2d}")
print(f"Gray code encoding of Conv2D: {gray_conv2d}")
print(f"Decoded Conv2D: {decoded_conv2d}")

binary_dropout = encode_layer_params('Dropout', dropout=0.3)
gray_dropout = binary_to_gray(binary_dropout)
decoded_dropout = decode_layer_params(gray_dropout)
print(f"Binary encoding of Dropout: {binary_dropout}")
print(f"Gray code encoding of Dropout: {gray_dropout}")
print(f"Decoded Dropout: {decoded_dropout}")

binary_dense = encode_layer_params('Dense', neurons=128, activation='relu')
gray_dense = binary_to_gray(binary_dense)
decoded_dense = decode_layer_params(gray_dense)
print(f"Binary encoding of Dense: {binary_dense}")
print(f"Gray code encoding of Dense: {gray_dense}")
print(f"Decoded Dense: {decoded_dense}")


Binary encoding of Conv2D: 00000000
Gray code encoding of Conv2D: 00000000
Decoded Conv2D: {'type': 'Conv2D', 'filters': 32, 'kernel_size': (3, 3), 'strides': 1, 'padding': 'same', 'activation': 'relu'}
Binary encoding of Dropout: 01101
Gray code encoding of Dropout: 01011
Decoded Dropout: {'type': 'Dropout', 'rate': 0.3}
Binary encoding of Dense: 1000100
Gray code encoding of Dense: 1100110
Decoded Dense: {'type': 'Dense', 'units': 128, 'activation': 'relu'}


# Binary Encoding of Building Blocks for Neural Network Architectures

This module implements the **binary encoding** of various building blocks that make up the architecture of a neural network. Each building block (layers like Conv2D, MaxPooling, Dense, etc.) is represented in binary format based on its key parameters (number of filters, kernel size, number of neurons, activations, etc.). This encoding is useful for optimization processes like **NSGA-III** and later conversion to **Gray code**.

### Function `encode_layer_params`

This function generates the binary representation of the parameters of a given layer, depending on the type of layer and its specific parameters.

### Function Parameters

- `layer_type`: (str) The type of layer. It can be one of the following:
  - `Conv2D`
  - `BatchNorm`
  - `MaxPooling`
  - `Dropout`
  - `Dense`
  - `Flatten`
  
- `filters`: (int) [Optional] Number of filters for `Conv2D` layers. Valid options: `32`, `30`, `16`, `8`.

- `kernel_size`: (tuple) [Optional] Kernel size for `Conv2D`. Currently, only `(3, 3)` is supported.

- `strides`: (int) [Optional] Stride used in `Conv2D` or `MaxPooling`. Valid options: `1` and `2`.

- `padding`: (str) [Optional] Padding type in `Conv2D`. Currently, only `'same'` is supported.

- `dropout`: (float) [Optional] Dropout rate for `Dropout` layers. Valid options: `0.2`, `0.3`, `0.5`.

- `neurons`: (int) [Optional] Number of neurons in `Dense` layers. Valid options: `256`, `128`, `32`.

- `activation`: (str) [Optional] Activation function for `Conv2D` or `Dense` layers. Valid options: `'ReLU'`, `'LeakyReLU'`, `'Sigmoid'`.

### Return

- `binary_representation`: (str) A string of bits representing the layer parameters in binary format.

## Layer Type Encoding

Each layer type is represented by a specific sequence of bits. The layer types and their corresponding encodings are:

- **Conv2D**: `'000'`
- **BatchNorm**: `'001'`
- **MaxPooling**: `'010'`
- **Dropout**: `'011'`
- **Dense**: `'100'`
- **Flatten**: `'101'`

## Parameters Encoded by Layer Type

### Conv2D
The `Conv2D` layer includes the following encoded parameters:

- **Number of filters**:
  - `32`: `'00'`
  - `30`: `'01'`
  - `16`: `'10'`
  - `8`: `'11'`

- **Kernel size**:
  - `(3, 3)`: `'0'` (only this size is supported for now)

- **Stride**:
  - `1`: `'0'`
  - `2`: `'1'`

- **Padding**:
  - `'same'`: `'0'`

- **Activation function**:
  - `ReLU`: `'00'`
  - `LeakyReLU`: `'01'`

### BatchNorm
The `BatchNorm` layer has no additional parameters, only its layer type encoding (`'001'`).

### MaxPooling
The `MaxPooling` layer has a single encoded parameter:

- **Stride**:
  - `1`: `'0'`
  - `2`: `'1'`

### Dropout
The `Dropout` layer has the following encoded dropout rates:

- **Dropout Rate**:
  - `0.2`: `'00'`
  - `0.3`: `'01'`
  - `0.5`: `'10'`

### Dense
The `Dense` layer has the following encoded parameters:

- **Number of neurons**:
  - `256`: `'00'`
  - `128`: `'01'`
  - `32`: `'10'`

- **Activation function**:
  - `ReLU`: `'00'`
  - `LeakyReLU`: `'01'`
  - `Sigmoid`: `'10'`

### Flatten
The `Flatten` layer has no additional parameters, only its layer type encoding (`'101'`).

## Gray Code Conversion

In addition to binary encoding, this module supports **Gray code** conversion for efficient optimization, where only one bit changes between consecutive values.

### Function `binary_to_gray`

This function converts a binary string to Gray code using the following logic:
- The first bit in Gray code is the same as the first bit in binary.
- For each subsequent bit, the Gray code bit is the XOR of the current binary bit and the previous binary bit.

### Function Parameters

- `binary_str`: (str) A string representing a binary number.

### Return

- `gray_str`: (str) A string representing the corresponding Gray code.

### Function `gray_to_binary`

This function converts a Gray code string back to its binary equivalent using the following logic:
- The first bit of the binary string is the same as the first bit of the Gray code.
- For each subsequent bit, the binary bit is the XOR of the previous binary bit and the current Gray code bit.

### Function Parameters

- `gray_str`: (str) A string representing a Gray code number.

### Return

- `binary_str`: (str) A string representing the corresponding binary number.




In [19]:
# Función para eliminar campos adicionales como 'padding' y 'kernel_size'
def clean_decoded_model(model_dict):
    # Recorrer cada capa del modelo decodificado
    for layer in model_dict['layers']:
        # Remover 'kernel_size' y 'padding' si están presentes
        if 'kernel_size' in layer:
            del layer['kernel_size']
        if 'padding' in layer:
            del layer['padding']
    return model_dict

# Función para decodificar la arquitectura del modelo a partir de Gray code
def decode_model_architecture(encoded_string_gray):
    model_dict = {'layers': []}
    index = 0
    
    while index < len(encoded_string_gray):
        # Extraer el resto del Gray code
        remaining_gray_code = encoded_string_gray[index:]
        
        # Decodificar una capa completa usando decode_layer_params
        decoded_layer = decode_layer_params(remaining_gray_code)
        
        # Agregar la capa decodificada al modelo
        model_dict['layers'].append(decoded_layer)
        
        # Obtener los parámetros relevantes de la capa (sin 'type')
        layer_type = decoded_layer['type']
        layer_params = {
            'filters': decoded_layer.get('filters'),
            'strides': decoded_layer.get('strides'),
            'dropout': decoded_layer.get('rate'),
            'neurons': decoded_layer.get('units'),
            'activation': decoded_layer.get('activation')
        }
        
        # Obtener la longitud de la codificación binaria para esta capa
        binary_code_for_layer = encode_layer_params(layer_type, **layer_params)
        gray_code_for_layer = binary_to_gray(binary_code_for_layer)
        
        # Avanzar el índice según la longitud de la capa decodificada
        index += len(gray_code_for_layer)
    
    return model_dict

# Función para codificar la arquitectura del modelo a Gray code
def encode_model_architecture(model_dict):
    encoded_layers_bin = []
    encoded_layers_gray = []
    
    # Codificar cada capa individualmente
    for layer in model_dict['layers']:
        binary_encoding = encode_layer_params(
            layer_type=layer['type'],
            filters=layer.get('filters'),
            strides=layer.get('strides'),
            dropout=layer.get('rate'),
            neurons=layer.get('units'),
            activation=layer.get('activation', '').lower()
        )
        encoded_layers_bin.append(binary_encoding)
        
        # Convertir a Gray code
        gray_encoding = binary_to_gray(binary_encoding)
        encoded_layers_gray.append(gray_encoding)
    
    # Concatenar todas las codificaciones en una sola cadena
    final_encoding_bin = ''.join(encoded_layers_bin)
    final_encoding_gray = ''.join(encoded_layers_gray)
    
    print(f"Binary Encoding: {final_encoding_bin}")
    print(f"Gray Code Encoding: {final_encoding_gray}")
    
    return final_encoding_gray

# Función de verificación
def verify_model_architecture(original_model):
    # Codificar el modelo original
    encoded_model = encode_model_architecture(original_model)
    
    # Decodificar el modelo desde el Gray code
    decoded_model = decode_model_architecture(encoded_model)
    
    # Limpiar el modelo decodificado para quitar 'kernel_size' y 'padding'
    decoded_model_cleaned = clean_decoded_model(decoded_model)
    
    # Verificar si el modelo original y decodificado son iguales
    if original_model == decoded_model_cleaned:
        print("Modelo original y decodificado son iguales.")
        return True
    else:
        print("Original Model:")
        print(original_model)
        print("\nDecoded Model (cleaned):")
        print(decoded_model_cleaned)
        return False

# Ejemplo de uso
model_example = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}

# Verificar el modelo
print(verify_model_architecture(model_example))

# Construcción del modelo en TensorFlow desde el diccionario decodificado
# Construcción del modelo en TensorFlow desde el diccionario decodificado
def build_tf_model_from_dict(model_dict):
    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
    
    model = Sequential()
    
    for layer in model_dict['layers']:
        if layer['type'] == 'Conv2D':
            model.add(Conv2D(filters=layer['filters'], kernel_size=(3, 3), strides=layer['strides'], padding='same', activation=layer['activation']))
        elif layer['type'] == 'BatchNorm':
            model.add(BatchNormalization())
        elif layer['type'] == 'MaxPooling':
            model.add(MaxPooling2D(pool_size=(2, 2), strides=layer['strides'], padding='same'))
        elif layer['type'] == 'Flatten':
            model.add(Flatten())
        elif layer['type'] == 'Dense':
            model.add(Dense(units=layer['units'], activation=layer['activation']))
        elif layer['type'] == 'Dropout':
            model.add(Dropout(rate=layer['rate']))
    
    # Aquí defines el tamaño de entrada para que el modelo se construya
    model.build(input_shape=(None, 28, 28, 3))  # Ajusta esto al tamaño de tu entrada
    
    return model



Binary Encoding: 0000000000101011011000000011111001110
Gray Code Encoding: 0000000000101111111100000010001101001
Modelo original y decodificado son iguales.
True


In [20]:
# Ejemplo 1: build_CNN_LF_model
model_CNN_LF = {
    "layers": [
        {"type": "Conv2D", "filters": 30, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},  # Revisar 'filters'
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.3},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 2: build_reduced_model
model_reduced = {
    "layers": [
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "leaky_relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 3: build_Spectro_CNN_model
model_spectro_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 4: Simple Conv2D model
model_simple_conv = {
    "layers": [
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 5: Simple Dense model
model_dense_only = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 256, "activation": "sigmoid"}
    ]
}

# Ejemplo 6: Small CNN model with Dropout
model_small_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 7: Deep CNN model
model_deep_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 8: Basic Dense with Dropout
model_dense_dropout = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 32, "activation": "tanh"}
    ]
}

model_with_depthwise = {
    "layers": [
        {"type": "DepthwiseConv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Dense", "units": 128, "activation": "sigmoid"},
        {"type": "Dense", "units": 1, "activation": "tanh"}
    ]
}
model_with_global_avg_pooling = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "GlobalAveragePooling2D"},
        {"type": "Dense", "units": 64, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}


In [21]:
# Diccionarios para la codificación/decodificación
layer_type_options = {
    'Conv2D': '000', 'BatchNorm': '001', 'MaxPooling': '010', 
    'Dropout': '011', 'Dense': '100', 'Flatten': '101'
}
filter_options = {32: '00', 30: '01', 16: '10', 8: '11'}
stride_options = {1: '0', 2: '1'}
dropout_options = {0.2: '00', 0.3: '01', 0.4: '10', 0.5: '11'}
neuron_options = {256: '00', 128: '01', 32: '10', 1: '11'}
activation_options = {'relu': '00', 'leaky_relu': '01', 'sigmoid': '10', 'tanh': '11'}

# Función para validar si los valores del modelo son válidos
def validate_model_layers(model):
    for layer in model['layers']:
        layer_type = layer['type']
        
        if layer_type == 'Conv2D':
            filters = layer.get('filters')
            strides = layer.get('strides')
            activation = layer.get('activation')

            if filters not in filter_options:
                print(f"Invalid filters value in Conv2D layer: {filters}")
                return False
            if strides not in stride_options:
                print(f"Invalid strides value in Conv2D layer: {strides}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in Conv2D layer: {activation}")
                return False
        
        elif layer_type == 'Dense':
            units = layer.get('units')
            activation = layer.get('activation')

            if units not in neuron_options:
                print(f"Invalid units value in Dense layer: {units}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in Dense layer: {activation}")
                return False

        elif layer_type == 'Dropout':
            rate = layer.get('rate')
            if rate not in dropout_options:
                print(f"Invalid dropout rate value: {rate}")
                return False
        
        elif layer_type == 'MaxPooling':
            strides = layer.get('strides')
            if strides not in stride_options:
                print(f"Invalid strides value in MaxPooling layer: {strides}")
                return False
        
        elif layer_type == 'Flatten':
            # No se necesitan validaciones adicionales para Flatten
            pass

        elif layer_type == 'BatchNorm':
            # No se necesitan validaciones adicionales para BatchNorm
            pass

    return True


# Función para verificar y construir el modelo
def run_verification_and_build(model):
    if validate_model_layers(model):
        print("Model is valid. Running verification and building the model:")
        # Verificación del modelo
        if verify_model_architecture(model):
            print("Model verification passed.")
        else:
            print("Model verification failed.")
        
        # Construcción del modelo en TensorFlow desde el diccionario decodificado
        decoded_model_dict = decode_model_architecture(encode_model_architecture(model))
        tf_model = build_tf_model_from_dict(decoded_model_dict)

        # Mostrar la estructura del modelo de TensorFlow
        try:
            tf_model.summary()
        except Exception as e:
            print(f"Error building model: {e}")
    else:
        print("Model is invalid. Skipping verification and build.")



In [22]:

# Verificación de los modelos
print("Verifications:")

print("\nModel 1: CNN_LF")
run_verification_and_build(model_CNN_LF)  # Ejemplo 1

print("\nModel 2: Reduced")
run_verification_and_build(model_reduced)  # Ejemplo 2

print("\nModel 3: Spectro CNN")
run_verification_and_build(model_spectro_CNN)  # Ejemplo 3

print("\nModel 4: Simple Conv2D")
run_verification_and_build(model_simple_conv)  # Ejemplo 4

print("\nModel 5: Dense Only")
run_verification_and_build(model_dense_only)  # Ejemplo 5

print("\nModel 6: Small CNN")
run_verification_and_build(model_small_CNN)  # Ejemplo 6

print("\nModel 7: Deep CNN")
run_verification_and_build(model_deep_CNN)  # Ejemplo 7

print("\nModel 8: Dense with Dropout")
run_verification_and_build(model_dense_dropout)  # Ejemplo 8

print("\nModel with DepthwiseConv2D")
run_verification_and_build(model_with_depthwise)

print("\nModel with GlobalAveragePooling2D")
run_verification_and_build(model_with_global_avg_pooling)

Verifications:

Model 1: CNN_LF
Model is valid. Running verification and building the model:
Binary Encoding: 00001000011000010101000100000110000101011011000000011011001110
Gray Code Encoding: 00001100010100010111000110000101000101111111100000010111101001
Modelo original y decodificado son iguales.
Model verification passed.
Binary Encoding: 00001000011000010101000100000110000101011011000000011011001110
Gray Code Encoding: 00001100010100010111000110000101000101111111100000010111101001
Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 28, 28, 30)        840       
                                                                 
 dropout_28 (Dropout)        (None, 28, 28, 30)        0         
                                                                 
 batch_normalization_28 (Bat  (None, 28, 28, 30)       120       
 chNormalization)        

KeyError: 'DepthwiseConv2D'